In [9]:
# Calculate all the ways to play
def winner_cells(board):
    for line in [
        [0, 1, 2],
        [3, 4, 5],
        [6, 7, 8],
        [0, 3, 6],
        [1, 4, 7],
        [2, 5, 8],
        [0, 4, 8],
        [2, 4, 6],
    ]:
        if board[line[0]] == board[line[1]] == board[line[2]] != 0:
            return line[0], line[2]  # Return only the first and last cells
    return None


def generate_game_states(board, move_count, game_string, game_states):
    win_cells = winner_cells(board)
    if move_count == 9 or win_cells:
        if win_cells:
            game_string += "9" + "".join(map(str, win_cells))
        game_states.append(game_string)
        return

    current_player = 1 if move_count % 2 == 0 else 2
    for i in range(9):
        if board[i] == 0:
            new_board = list(board)
            new_board[i] = current_player
            new_game_string = game_string + str(i)
            generate_game_states(
                new_board, move_count + 1, new_game_string, game_states
            )


empty_board = [0] * 9
game_states = []
generate_game_states(empty_board, 0, "", game_states)

print(f"Generated {len(game_states)} ways to play Tic Tac Toe.")

Generated 255168 ways to play Tic Tac Toe.


In [97]:
# Create mapping between games and position
import json

TILES_ACROSS = 576
TILES_DOWN = 443


def map_game_states_to_tiles(game_states):
    tile_positions = {}
    for idx, game_state in enumerate(game_states):
        row = idx // TILES_ACROSS
        col = idx % TILES_ACROSS
        tile_positions[game_state] = (row, col)
    return tile_positions


tile_positions = map_game_states_to_tiles(game_states)

for game_state, position in tile_positions.items():
    tile_positions[game_state] = list(position)

with open("tile_positions.json", "w") as file:
    json.dump(tile_positions, file)

print("Tile positions have been written to tile_positions.json.")

Tile positions have been written to tile_positions.json.


In [98]:
# Functions that draw the board and pieces
from PIL import Image, ImageDraw, ImageFont


def draw_X(draw, x, y, cell_size, padding):
    # Draw the X
    bitmap = [
        [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1],
        [0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0],
        [0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0],
        [0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0],
        [0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0],
        [0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0],
        [0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0],
        [0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0],
        [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1],
    ]
    scale = 1

    # Draw the "X" by scaling the bitmap
    for row in range(13):
        for col in range(13):
            if bitmap[row][col]:
                draw.point(
                    (
                        x + col + padding,
                        y + row + padding,
                    ),
                    fill="black",
                )


def draw_O(draw, x, y, cell_size, padding):
    bounding_box = [
        x + padding,
        y + padding,
        x + cell_size - padding,
        y + cell_size - padding,
    ]
    draw.ellipse(bounding_box, outline="black", width=2)


def draw_board(board, player, winning_cells=None):
    size = (64, 64)
    padding = 8  # Board padding
    cell_size = (size[0] - 2 * padding) // 3
    image = Image.new("RGB", size, "white")
    draw = ImageDraw.Draw(image)

    # Draw grid lines
    for i in range(1, 3):
        draw.line(
            [
                padding + i * cell_size,
                padding,
                padding + i * cell_size,
                size[1] - padding,
            ],
            fill="black",
        )
        draw.line(
            [
                padding,
                padding + i * cell_size,
                size[0] - padding,
                padding + i * cell_size,
            ],
            fill="black",
        )

    font_path = "./fonts/retro_computer_personal_use.ttf"
    font_size = 15
    font = ImageFont.truetype(font_path, font_size)
    # font = ImageFont.load_default()
    cell_padding = 2  # Adjust this value to change the padding

    for i, cell in enumerate(board):
        row = i // 3
        col = i % 3
        x = padding + col * cell_size
        y = padding + row * cell_size
        if cell == "X":
            draw_X(draw, x, y, cell_size, cell_padding)
        elif cell == "O":
            draw_O(draw, x, y, cell_size, cell_padding)

    # Draw winning line if winning_cells provided
    if winning_cells:
        x1 = padding + (winning_cells[0] % 3 + 0.5) * cell_size
        y1 = padding + (winning_cells[0] // 3 + 0.5) * cell_size
        x2 = padding + (winning_cells[1] % 3 + 0.5) * cell_size
        y2 = padding + (winning_cells[1] // 3 + 0.5) * cell_size
        draw.line([x1, y1, x2, y2], fill="red", width=3)

    return image

In [100]:
# Generate APNG sequence
from PIL import ImageSequence


def create_apng(move_string, save_path):
    frames = []
    board = ["_"] * 9
    player = "X"
    winning_cells = None

    # Append the initial blank state
    frames.append(draw_board(board, player))

    # Iterate through the moves
    for i in range(len(move_string)):
        if i < len(move_string) - 2 and move_string[i] == "9":
            winning_cells = [int(move_string[i + 1]), int(move_string[i + 2])]
            break  # Stop iteration after finding the '9' character
        move_idx = int(move_string[i])
        if 0 <= move_idx < 9:
            board[move_idx] = player
            frames.append(draw_board(board, player))
            player = "O" if player == "X" else "X"

    # Append the final state with the winning line if there was a win
    if winning_cells:
        frames.append(draw_board(board, player, winning_cells))

    # Save as an APNG
    frames[0].save(
        save_path + ".png",
        save_all=True,
        append_images=frames[1:],
        duration=500,
        loop=0,
    )


move_string = "0123456926"
create_apng(move_string, "tic_tac_toe")

# Display APNG
from IPython.display import Image as IPImage

IPImage(filename="tic_tac_toe.png")

In [95]:
# Crank out all of the APNGS
def format_tile_position(arr):
    return f"tile_{arr[0]}_{arr[1]}"


tile_directory = "./tiles/0"

for i, move_string in enumerate(game_states):
    tile_name = f"{tile_directory}/{format_tile_position(tile_positions[move_string])}"
    create_apng(move_string, tile_name)